# IBM Applied Data Science Capstone Course by Coursera

## Week 5 Final Report

### Opening a New Shopping Mall in Delhi, India

* Build a dataframe of neighborhoods in Delhi, India by web scraping the data from wikipedia page
* Get the geographical coordinates of the neighborhoods
* Obtain the venue data for the neighborhoods from Foursquare API
* Explore and cluster the neighborhoods
* Select the best cluster for opening a new shopping mall

### 1. Importing libraries

In [39]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scraping data from Wikipedia page into a DataFrame

In [27]:
# sending the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Delhi").text

In [28]:
# parsing data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [29]:
# creating a list to store neighborhood data
neighborhoodList = []

In [30]:
# appending the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [31]:
print(neighborhoodList)

['Neighbourhoods of Delhi', 'Ashok Nagar (Delhi)', 'Ashok Vihar', 'Ashram Chowk', 'Babarpur', 'Badarpur, Delhi', 'Bali Nagar', 'Bawana', 'Ber Sarai', 'Bhajanpura', 'Chanakyapuri', 'Chandni Chowk', 'Chittaranjan Park', 'Civil Lines, Delhi', 'Connaught Place, New Delhi', 'Dabri, New Delhi', 'Dariba Kalan', 'Daryaganj', 'Dashrath Puri', 'Dayanand Colony', 'Defence Colony', 'Delhi Cantonment', 'Derawal Nagar', 'Dhaula Kuan', 'Dilshad Colony', 'Dilshad Garden', 'Dwarka, Delhi', 'East Patel Nagar', 'Gadaipur, Mehrauli, New Delhi', 'Gandhi Nagar, Delhi', 'Geetanjali Enclave', 'Ghitorni', 'Gole Market', 'Golf Links, New Delhi', 'Govindpuri', 'Greater Kailash', 'Green Park, Delhi', 'Gulabi Bagh', 'Gulmohar Park', 'Hauz Khas', 'Inder Puri', 'Janakpuri', 'Jangpura', 'Jia Sarai', 'Kabir Nagar, New Delhi', 'Kailash Colony', 'Kamla Nagar, New Delhi', 'Kapasheda Border, Delhi', 'Karol Bagh', 'Keshav Puram', 'Khaira, Delhi', 'Khanpur, Delhi', 'Khari Baoli', 'Kingsway Camp', 'Kirti Nagar', 'Kotla Mubar

In [32]:
# Removing "Neighbourhoods of Delhi" because it doesn't represents specific place
# and is just a hyperlink of neighbourhoods of delhi
neighborhoodList.remove('Neighbourhoods of Delhi')

In [33]:
# creating a new DataFrame from the list
del_df = pd.DataFrame({"Neighborhood": neighborhoodList})

del_df.head()

,Neighborhood
0,Ashok Nagar (Delhi)
1,Ashok Vihar
2,Ashram Chowk
3,Babarpur
4,"Badarpur, Delhi"


In [34]:
# printing the number of rows of the dataframe
del_df.shape

(140, 1)

### 3. Get the geographical coordinates

In [40]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Delhi, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [41]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in del_df["Neighborhood"].tolist() ]

In [42]:
coords

[[28.692230000000052, 77.30124000000006],
 [28.69037000000003, 77.17609000000004],
 [28.710597501792023, 77.32696517369723],
 [28.50738000000007, 77.30346000000003],
 [28.50738000000007, 77.30346000000003],
 [28.652234222889238, 77.12939224396462],
 [28.800590000000057, 77.03473000000008],
 [28.549540000000036, 77.18167000000005],
 [28.699880000000064, 77.25906000000003],
 [28.595060000000046, 77.18573000000004],
 [28.656270000000063, 77.23232000000007],
 [28.538400000000024, 77.24832000000004],
 [28.67671000000007, 77.21767000000006],
 [28.633940000000052, 77.21968000000004],
 [28.60761000000008, 77.08714000000003],
 [28.65457890544559, 77.23339989939495],
 [28.62832000000003, 77.24727000000007],
 [28.605920000000026, 77.08529000000004],
 [28.560590000000047, 77.24678000000006],
 [28.57298000000003, 77.23357000000004],
 [28.591510000000028, 77.12945000000008],
 [28.699110000000076, 77.19105000000008],
 [28.594857177133914, 77.16729160908383],
 [28.684700000000078, 77.32774000000006],


In [43]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [44]:
# merge the coordinates into the original dataframe
del_df['Latitude'] = df_coords['Latitude']
del_df['Longitude'] = df_coords['Longitude']

In [45]:
# check the neighborhoods and the coordinates
print(del_df.shape)
del_df

(140, 3)


,Neighborhood,Latitude,Longitude
0,Ashok Nagar (Delhi),28.692230,77.301240
1,Ashok Vihar,28.690370,77.176090
2,Ashram Chowk,28.710598,77.326965
3,Babarpur,28.507380,77.303460
4,"Badarpur, Delhi",28.507380,77.303460
5,Bali Nagar,28.652234,77.129392
6,Bawana,28.800590,77.034730
7,Ber Sarai,28.549540,77.181670
8,Bhajanpura,28.699880,77.259060
9,Chanakyapuri,28.595060,77.185730


In [46]:
# save the DataFrame as CSV file
del_df.to_csv("del_df.csv", index=False)

### 4. Creating a map of Delhi with neighborhoods superimposed on top

In [47]:
# get the coordinates of Delhi
address = 'Delhi, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi, India 28.6517178, 77.2219388.


In [48]:
# create map of Delhi using latitude and longitude values
map_del = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(del_df['Latitude'], del_df['Longitude'], del_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_del)  
    
map_del

In [49]:
# save the map as HTML file
map_del.save('map_del.html')

### 5. Using the Foursquare API to explore the neighborhoods

In [75]:
# define Foursquare Credentials and Version
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret


#### Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [52]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(del_df['Latitude'], del_df['Longitude'], del_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [53]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6173, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Ashok Nagar (Delhi),28.69223,77.30124,AFM PVT LTD,28.704770,77.309608,Tourist Information Center
1,Ashok Nagar (Delhi),28.69223,77.30124,yamuna vihar,28.689816,77.283876,Park
2,Ashok Nagar (Delhi),28.69223,77.30124,Shivaji park,28.682657,77.285503,Park
3,Ashok Nagar (Delhi),28.69223,77.30124,Mansarover Park Metro Station,28.675370,77.300932,Train Station
4,Ashok Vihar,28.69037,77.17609,Major Dhyan Chand Sports Complex,28.684029,77.167487,Athletics & Sports


#### Checking how many venues were returned for each neighorhood

In [54]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Ashok Nagar (Delhi),4,4,4,4,4,4
Ashok Vihar,23,23,23,23,23,23
Ashram Chowk,5,5,5,5,5,5
Babarpur,5,5,5,5,5,5
"Badarpur, Delhi",5,5,5,5,5,5
Bali Nagar,51,51,51,51,51,51
Bawana,2,2,2,2,2,2
Ber Sarai,94,94,94,94,94,94
Bhajanpura,5,5,5,5,5,5


#### Finding out how many unique categories can be curated from all the returned venues

In [55]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 212 uniques categories.


In [56]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Tourist Information Center', 'Park', 'Train Station',
       'Athletics & Sports', 'Asian Restaurant', 'Sandwich Place',
       'Pizza Place', 'Snack Place', 'Indian Restaurant',
       'Department Store', 'South Indian Restaurant',
       'Fast Food Restaurant', 'Market', 'Coffee Shop', 'Dessert Shop',
       'Basketball Court', 'Light Rail Station', 'Bakery', 'ATM',
       'Print Shop', 'Indian Sweet Shop', 'American Restaurant',
       'Donut Shop', 'Diner', 'Hookah Bar', 'BBQ Joint', 'Café', 'Hotel',
       'Garden Center', 'Sports Bar', 'Furniture / Home Store', 'Pub',
       'Multiplex', 'Shopping Mall', 'Bar', 'Fried Chicken Joint',
       'Chinese Restaurant', 'Gourmet Shop', 'Garden',
       'Convenience Store', 'Cafeteria', 'Clothing Store', 'Playground',
       'Art Gallery', 'Lounge', 'Mediterranean Restaurant',
       'Tibetan Restaurant', 'Tea Room', 'Ice Cream Shop', 'Beer Garden'],
      dtype=object)

In [57]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### 6. Analyzing Each Neighborhood

In [58]:
# one hot encoding
del_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
del_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [del_onehot.columns[-1]] + list(del_onehot.columns[:-1])
del_onehot = del_onehot[fixed_columns]

print(del_onehot.shape)
del_onehot.head()

(6173, 213)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,High School,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Punjabi Restaurant,Racetrack,Rest Area,Restaurant,River,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Women's Store,Yoga Studio,Zoo
0,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [59]:
del_grouped = del_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(del_grouped.shape)
del_grouped

(139, 213)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,High School,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Punjabi Restaurant,Racetrack,Rest Area,Restaurant,River,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Women's Store,Yoga Studio,Zoo
0,Ashok Nagar (Delhi),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [60]:
len(del_grouped[del_grouped["Shopping Mall"] > 0])

41

#### Creating a new dataframe for Shopping Mall data only

In [61]:
del_mall = del_grouped[["Neighborhoods","Shopping Mall"]]

In [62]:
del_mall.head()

,Neighborhoods,Shopping Mall
0,Ashok Nagar (Delhi),0.0
1,Ashok Vihar,0.0
2,Ashram Chowk,0.0
3,Babarpur,0.0
4,"Badarpur, Delhi",0.0


### 7. Clustering Neighborhoods

In [63]:
# set number of clusters
kclusters = 3

del_clustering = del_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(del_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 1])

In [64]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
del_merged = del_mall.copy()

# add clustering labels
del_merged["Cluster Labels"] = kmeans.labels_

In [65]:
del_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
del_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Ashok Nagar (Delhi),0.0,0
1,Ashok Vihar,0.0,0
2,Ashram Chowk,0.0,0
3,Babarpur,0.0,0
4,"Badarpur, Delhi",0.0,0


In [66]:
# merging dataframes to add latitude/longitude for each neighborhood
del_merged = del_merged.join(del_df.set_index("Neighborhood"), on="Neighborhood")

print(del_merged.shape)
del_merged.head() # check the last columns!

(139, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Ashok Nagar (Delhi),0.0,0,28.692230,77.301240
1,Ashok Vihar,0.0,0,28.690370,77.176090
2,Ashram Chowk,0.0,0,28.710598,77.326965
3,Babarpur,0.0,0,28.507380,77.303460
4,"Badarpur, Delhi",0.0,0,28.507380,77.303460


In [67]:
# sorting the results by Cluster Labels
print(del_merged.shape)
del_merged.sort_values(["Cluster Labels"], inplace=True)
del_merged

(139, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Ashok Nagar (Delhi),0.000000,0,28.692230,77.301240
90,Paharganj,0.000000,0,28.645960,77.214920
89,Old Delhi,0.000000,0,28.654340,77.232580
88,Okhla,0.000000,0,28.532470,77.278390
87,Nizamuddin West,0.000000,0,28.588650,77.244860
86,Nizamuddin East,0.000000,0,28.602462,77.265791
85,Nigambodh Ghat,0.000000,0,28.664710,77.236330
84,New Moti Bagh,0.022472,0,28.580997,77.181823
83,New Friends Colony,0.000000,0,28.578120,77.269990
82,New Delhi,0.000000,0,28.630950,77.217210


#### Finally, visualising the resulting clusters

In [68]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(del_merged['Latitude'], del_merged['Longitude'], del_merged['Neighborhood'], del_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [69]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examining Clusters

#### Cluster 0

In [70]:
del_merged.loc[del_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Ashok Nagar (Delhi),0.000000,0,28.692230,77.301240
90,Paharganj,0.000000,0,28.645960,77.214920
89,Old Delhi,0.000000,0,28.654340,77.232580
88,Okhla,0.000000,0,28.532470,77.278390
87,Nizamuddin West,0.000000,0,28.588650,77.244860
86,Nizamuddin East,0.000000,0,28.602462,77.265791
85,Nigambodh Ghat,0.000000,0,28.664710,77.236330
84,New Moti Bagh,0.022472,0,28.580997,77.181823
83,New Friends Colony,0.000000,0,28.578120,77.269990
82,New Delhi,0.000000,0,28.630950,77.217210


#### Cluster 1

In [71]:
del_merged.loc[del_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
71,Moti Nagar (New Delhi),0.054545,1,28.656720,77.129600
51,Kingsway Camp,0.043478,1,28.711690,77.201970
125,Shivaji Place,0.053571,1,28.652650,77.121390
94,Pandav Nagar,0.055556,1,28.614580,77.275740
129,Tilak Nagar (Delhi),0.047619,1,28.640890,77.089220
66,Mayur Vihar,0.050000,1,28.612795,77.288501
5,Bali Nagar,0.058824,1,28.652234,77.129392
132,Vasant Kunj,0.047619,1,28.531520,77.150200
57,Laxmibai Nagar,0.033333,1,28.578150,77.206180
9,Chanakyapuri,0.026316,1,28.595060,77.185730


#### Cluster 2

In [72]:
del_merged.loc[del_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
20,Delhi Cantonment,0.125000,2,28.59151,77.12945
107,"Sadar Bazaar, Delhi",0.111111,2,28.59028,77.12014
46,"Kapasheda Border, Delhi",0.111111,2,28.52163,77.08645
105,"Rohini, Delhi",0.111111,2,28.73356,77.10401
56,Laxmi Nagar (Delhi),0.125000,2,28.63875,77.27592
97,Pitam Pura,0.088235,2,28.69590,77.13725
52,Kirti Nagar,0.088235,2,28.64821,77.14273


#### OBSERVATIONS
##### Most of the shopping malls are concentrated in the central area of Delhi, with the highest number in cluster 2 and moderate number in cluster 1. On the other hand, cluster 0 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.